In [ ]:
import pandas as pd
import snowflake.connector
import yfinance as yf
import numpy as np
import datetime
import time
from dotenv import load_dotenv
import os

load_dotenv()

conn_params = {
    'user': os.getenv('SNOWFLAKE_USER'),
    'password': os.getenv('SNOWFLAKE_PASSWORD'),
    'account': os.getenv('SNOWFLAKE_ACCOUNT'),
    'warehouse': os.getenv('SNOWFLAKE_WAREHOUSE'),
    'database': os.getenv('SNOWFLAKE_DATABASE'),
    'schema': os.getenv('SNOWFLAKE_SCHEMA'),
    'role': os.getenv('SNOWFLAKE_ROLE')
}

In [6]:
def connect_to_snowflake(params):
    try:
        conn = snowflake.connector.connect(
            user=params['user'],
            password=params['password'],
            account=params['account'],
            warehouse=params['warehouse'],
            database=params['database'],
            schema=params['schema'],
            role=params['role']  # Specify the role if needed
        )
        print("Connected to Snowflake successfully!")
        return conn
    except Exception as e:
        print(f"Error connecting to Snowflake: {e}")
        return None

In [7]:
conn = connect_to_snowflake(conn_params)

if conn is None:
    print("Failed to connect to Snowflake. Exiting.")


Connected to Snowflake successfully!


## Table updation with YFINANCE

In [ ]:


def create_table_in_snowflake(conn, table_name="LARGE_COMPANY_DATA_FINAL", schema="OPPORTUNITY_ANALYSIS.MARKET_ANALYSIS"):
    create_query = f"""
    CREATE OR REPLACE TABLE {schema}.{table_name} (
        ID VARCHAR,
        COMPANY_NAME VARCHAR,
        INDUSTRY VARCHAR,
        SIZE VARCHAR,
        FOUNDED NUMBER,
        REGION VARCHAR,
        COUNTRY VARCHAR,
        LOCALITY VARCHAR,
        WEBSITE VARCHAR,
        LINKEDIN_URL VARCHAR,
        COMPANY_AGE NUMBER,
        SIZE_CATAGORY VARCHAR,
        PERFORMANCE_CATEGORY VARCHAR,
        PERFORMANCE_SCORE FLOAT,
        T_SYMBOL VARCHAR,
        TICKER VARCHAR,
        CURRENT_PRICE FLOAT,
        MARKET_CAP NUMBER,
        YEARLY_RETURN NUMBER,
        VOLATILITY VARCHAR
    );
    """
    try:
        cursor = conn.cursor()
        cursor.execute(create_query)
        print(f"Table {schema}.{table_name} created or replaced.")
    except Exception as e:
        print(f"Error creating table: {e}")
    finally:
        cursor.close()

def save_df_to_snowflake(df, table_name="LARGE_COMPANY_DATA_FINAL", database="OPPORTUNITY_ANALYSIS", schema="MARKET_ANALYSIS"):
    conn = connect_to_snowflake(conn_params)
    if conn is None:
        print("❌ Could not connect to Snowflake.")
        return

    try:
        conn.cursor().execute(f"USE DATABASE {database}")
        conn.cursor().execute(f"USE SCHEMA {schema}")

        df.columns = [col.upper() for col in df.columns]

        create_table_in_snowflake(conn, table_name, f"{database}.{schema}")

        # Write data using uppercase column names
        success, nchunks, nrows, _ = write_pandas(conn, df, table_name=table_name, schema=schema)

        if success:
            print(f"✅ Successfully inserted {nrows} rows into {database}.{schema}.{table_name}")
        else:
            print(f"❌ Failed to write DataFrame to Snowflake")

    except Exception as e:
        print(f"❌ Error writing DataFrame to Snowflake: {e}")
    finally:
        conn.close()

In [ ]:
def get_company_data():
    """Fetch company data with ticker symbols from Snowflake"""
    conn = connect_to_snowflake(conn_params)
    if conn is None:
        return None
    
    try:
        query = """
        SELECT * FROM LARGE_COMPANIES_WITH_TICKER
        WHERE T_SYMBOL IS NOT NULL
        """
        
        df = pd.read_sql(query, conn)
        conn.close()
        print(f"Retrieved {len(df)} companies with ticker symbols")
        return df
    except Exception as e:
        print(f"Error fetching company data: {e}")
        if conn:
            conn.close()
        return None

def get_yfinance_metrics(ticker_list):
    """Get relevant financial metrics for a list of tickers"""
    # Filter out None values
    ticker_list = [t for t in ticker_list if t]
    
    if not ticker_list:
        return pd.DataFrame()
    results = {}
    batch_size = 15
    for i in range(0, len(ticker_list), batch_size):
        batch = ticker_list[i:i+batch_size]
        print(f"Processing batch {i//batch_size + 1}/{(len(ticker_list) + batch_size - 1)//batch_size}")
        
        for ticker in batch:
            try:
                ticker_obj = yf.Ticker(ticker)
                info = ticker_obj.info
                hist = ticker_obj.history(period="1y")
                if len(hist) > 0:
                    start_price = hist['Close'].iloc[0]
                    current_price = hist['Close'].iloc[-1]
                    yearly_return = ((current_price - start_price) / start_price) * 100
                    daily_returns = hist['Close'].pct_change().dropna()
                    volatility = daily_returns.std() * 100 * np.sqrt(252)  # Annualized
                else:
                    yearly_return = None
                    volatility = None
                
                # Store the results
                results[ticker] = {
                    'ticker': ticker,
                    'current_price': current_price,
                    'market_cap': info.get('marketCap', None),
                    'yearly_return': yearly_return,
                    'volatility': volatility,
                }
                time.sleep(0.1)  # To avoid hitting the API too fast
                
            except Exception as e:
                print(f"Error processing ticker {ticker}: {e}")
                # Add the ticker to results with None values to maintain the record
                results[ticker] = {
                    'ticker': ticker,
                    'current_price': None,
                    'market_cap': None,
                    'yearly_return': None,
                    'volatility': None
                }
    
    metrics_df = pd.DataFrame.from_dict(results, orient='index')
    return metrics_df

def enhance_company_data_with_yfinance():
    company_df = get_company_data()
    
    if company_df is None or company_df.empty:
        print("No company data available. Exiting.")
        return

    ticker_symbols = company_df['T_SYMBOL'].unique().tolist()
    print(f"Found {len(ticker_symbols)} unique ticker symbols")
    
    metrics_df = get_yfinance_metrics(ticker_symbols)
    
    if metrics_df.empty:
        print("No metrics data retrieved. Exiting.")
        return
    
    enhanced_df = pd.merge(
        company_df,
        metrics_df,
        left_on='T_SYMBOL',
        right_on='ticker',
        how='left'
    )
    enhanced_df = enhanced_df.drop(columns=['ticker'])
    
    print(f"Enhanced dataset has {enhanced_df.shape[1]} columns and {enhanced_df.shape[0]} rows")
    
    # Write the enhanced data back to Snowflake
    save_df_to_snowflake(enhanced_df)
    
    return enhanced_df

In [10]:
en_df = enhance_company_data_with_yfinance()


Connected to Snowflake successfully!


C:\Users\yohan\AppData\Local\Temp\ipykernel_21096\657144211.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Retrieved 2007 companies with ticker symbols
Found 1782 unique ticker symbols
Processing batch 1/90
Processing batch 2/90
Processing batch 3/90


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CELG.R?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CELG.R&crumb=3JMyOGxXE8v


Error processing ticker CELG.R: 'NoneType' object has no attribute 'update'
Processing batch 4/90
Processing batch 5/90
Processing batch 6/90
Processing batch 7/90
Processing batch 8/90
Processing batch 9/90
Processing batch 10/90
Processing batch 11/90
Processing batch 12/90
Processing batch 13/90
Processing batch 14/90
Processing batch 15/90
Processing batch 16/90
Processing batch 17/90
Processing batch 18/90
Processing batch 19/90
Processing batch 20/90
Processing batch 21/90
Processing batch 22/90
Processing batch 23/90
Processing batch 24/90
Processing batch 25/90
Processing batch 26/90
Processing batch 27/90
Processing batch 28/90
Processing batch 29/90
Processing batch 30/90
Processing batch 31/90
Processing batch 32/90
Processing batch 33/90
Processing batch 34/90
Processing batch 35/90
Processing batch 36/90
Processing batch 37/90
Processing batch 38/90
Processing batch 39/90
Processing batch 40/90
Processing batch 41/90
Processing batch 42/90
Processing batch 43/90
Processing

...

Error processing ticker CALX: 'NoneType' object has no attribute 'update'
Processing batch 58/90
Processing batch 59/90
Processing batch 60/90
Processing batch 61/90
Processing batch 62/90
Processing batch 63/90
Processing batch 64/90
Processing batch 65/90
Processing batch 66/90
Processing batch 67/90
Processing batch 68/90
Processing batch 69/90
Processing batch 70/90
Processing batch 71/90


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AACT.W?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AACT.W&crumb=3JMyOGxXE8v


Error processing ticker AACT.W: 'NoneType' object has no attribute 'update'
Processing batch 72/90
Processing batch 73/90
Processing batch 74/90
Error processing ticker SGRP: Too Many Requests. Rate limited. Try after a while.
Error processing ticker ITAN: Too Many Requests. Rate limited. Try after a while.
Processing batch 75/90
Error processing ticker MWA: Too Many Requests. Rate limited. Try after a while.
Error processing ticker MUSA: Too Many Requests. Rate limited. Try after a while.
Error processing ticker MYE: Too Many Requests. Rate limited. Try after a while.
Error processing ticker MYRG: Too Many Requests. Rate limited. Try after a while.
Error processing ticker MYGN: Too Many Requests. Rate limited. Try after a while.
Error processing ticker NBR: Too Many Requests. Rate limited. Try after a while.
Error processing ticker QQQM: Too Many Requests. Rate limited. Try after a while.
Error processing ticker NTRA: Too Many Requests. Rate limited. Try after a while.
Error processin

401 Client Error: Unauthorized for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ORN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ORN&crumb=Edge%3A+Too+Many+Requests


Error processing ticker ORN: 'NoneType' object has no attribute 'update'
Processing batch 84/90
Processing batch 85/90
Processing batch 86/90
Processing batch 87/90
Processing batch 88/90
Processing batch 89/90
Processing batch 90/90
Enhanced dataset has 19 columns and 2007 rows
Data saved to enhanced_company_data.csv


In [11]:
en_df.tail()

,ID,INDUSTRY,COMPANY_NAME,SIZE,FOUNDED,REGION,COUNTRY,LOCALITY,WEBSITE,LINKEDIN_URL,COMPANY_AGE,SIZE_CATAGORY,PERFORMANCE_CATEGORY,PERFORMANCE_SCORE,T_SYMBOL,current_price,market_cap,yearly_return,volatility
2002,9ThdbBO1CPzgl4CCs2uwRQU9DIH5,entertainment,roku inc.,1001-5000,2002,california,united states,san jose,roku.com,linkedin.com/company/roku,23,large,Steady Performer,99.0,ROKU,58.459999,8.531886e+09,-0.391889,60.906372
2003,PjZeL0lwqfBy3Mdz9JAWZwOOwIsb,consumer services,"rollins, inc.",10001+,1948,georgia,united states,atlanta,rollins.com,linkedin.com/company/rollins-inc.,77,large,Developing,60.0,ROL,55.779999,2.705754e+10,33.129381,22.339347
2004,O9KpsdPIquEYFfiVHJOgBQOrd6QS,insurance,root inc.,1001-5000,2015,ohio,united states,columbus,root.com,linkedin.com/company/rootinsurance,10,large,Steady Performer,93.5,ROOT,129.740005,1.972048e+09,152.265224,110.907070
2005,ePKif3bzXcOYABNSCwtZcAGLoUse,mechanical or industrial engineering,"roper technologies, inc.",10001+,1981,florida,united states,sarasota,ropertech.com,linkedin.com/company/roper-industries,44,large,Developing,66.0,ROP,557.239990,5.983922e+10,6.294555,21.307624
2006,Qhk28SodybHScQJ97BfuYwJUbTS0,retail,"ross stores, inc.",10001+,1982,california,united states,dublin,rossstores.com,linkedin.com/company/ross-stores,43,large,Developing,72.0,ROST,139.630005,4.591509e+10,7.067430,24.694364


In [12]:
save_df_to_snowflake(en_df)

Connected to Snowflake successfully!
Table OPPORTUNITY_ANALYSIS.MARKET_ANALYSIS.LARGE_COMPANY_DATA_FINAL created or replaced.
✅ Successfully inserted 2007 rows into OPPORTUNITY_ANALYSIS.MARKET_ANALYSIS.LARGE_COMPANY_DATA_FINAL


In [ ]:
import pandas as pd
import finnhub

#list of available exchanges

df=pd.read_html("https://docs.google.com/spreadsheets/d/1I3pBxjfXB056-g_JYf_6o3Rns3BV2kMGG1nCatb91ls/edit#gid=0")
df1=df[0]
exc=df1.loc[:,"A"].dropna()

exclist=[]
for i in exc:
    exclist.append(str(i))
exclist=exclist[1:] #take out "name" from the list

#retrieve tickers from every exchange available
tickers=[]
finnhub_client = finnhub.Client(api_key="c46qn9iad3iagvmhdk7g")
for exchange in exclist:
    listofdicts=finnhub_client.stock_symbols(exchange)
    for dicts in listofdicts:
        tickers.append(dicts['symbol'])

print("You just got a list of %s tickers worldwide." % len(tickers), tickers)

In [2]:
import time
import pandas as pd
import yfinance as yf
from get_all_tickers.get_tickers import get_tickers

tickers_list = get_tickers()

tickers_list = tickers_list[:50]

# Initialize an empty list to store the data.
data = []

for ticker in tickers_list:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info  # Retrieve the info dictionary from yfinance
        
        # Extract the company name and market capitalization.
        company_name = info.get('longName', 'N/A')
        market_cap = info.get('marketCap', 'N/A')
        
        # Append the data as a dictionary.
        data.append({
            'Ticker': ticker,
            'Company Name': company_name,
            'Market Cap': market_cap
        })
        
        # Pause briefly to respect rate limits.
        time.sleep(0.1)
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {e}")
        continue

# Create a DataFrame from the list of dictionaries.


c:\Users\yohan\anaconda3\envs\final_proj_1\Lib\site-packages\get_all_tickers\get_tickers.py:95: SyntaxWarning: invalid escape sequence '\.'
  df = df[~df['Symbol'].str.contains("\.|\^")]
c:\Users\yohan\anaconda3\envs\final_proj_1\Lib\site-packages\get_all_tickers\get_tickers.py:140: SyntaxWarning: invalid escape sequence '\.'
  df_filtered = df[~df['Symbol'].str.contains("\.|\^")]
c:\Users\yohan\anaconda3\envs\final_proj_1\Lib\site-packages\get_all_tickers\get_tickers.py:147: SyntaxWarning: invalid escape sequence '\.'
  df = df[~df['Symbol'].str.contains("\.|\^")]


ParserError: Error tokenizing data. C error: Expected 1 fields in line 9, saw 2


In [3]:
from pytickersymbols import PyTickerSymbols

stock_data = PyTickerSymbols()
german_stocks = stock_data.get_stocks_by_index('DAX')
uk_stocks = stock_data.get_stocks_by_index('FTSE 100')

print(list(uk_stocks))

[{'name': '3i Group PLC', 'symbol': 'III', 'country': 'United Kingdom', 'indices': ['FTSE 100'], 'industries': ['Banking & Investment Services', 'Investment Banking & Investment Services', 'Financials', 'Investment Management & Fund Operators'], 'symbols': [{'yahoo': 'IGQ5.F', 'google': 'FRA:IGQ5', 'currency': 'EUR'}, {'yahoo': 'TGOPY', 'google': 'OTCMKTS:TGOPY', 'currency': 'USD'}, {'yahoo': 'III.L', 'google': 'LON:III', 'currency': 'GBP'}, {'yahoo': 'IGQ.F', 'google': 'FRA:IGQ', 'currency': 'EUR'}, {'yahoo': 'TGOPF', 'google': 'OTCMKTS:TGOPF', 'currency': 'USD'}], 'metadata': {'founded': 1945, 'employees': 244}, 'isins': ['GB00B1YW4409'], 'akas': []}, {'name': 'Admiral Group PLC', 'symbol': 'ADM', 'country': 'United Kingdom', 'indices': ['FTSE 100'], 'industries': ['Insurance', 'Property & Casualty Insurance', 'Financials'], 'symbols': [{'yahoo': 'FLN.F', 'google': 'FRA:FLN', 'currency': 'EUR'}, {'yahoo': 'ADM.L', 'google': 'LON:ADM', 'currency': 'GBP'}, {'yahoo': 'AMIGY', 'google': 

In [5]:
from pytickersymbols import PyTickerSymbols

# Initialize the PyTickerSymbols object
stock_data = PyTickerSymbols()

# Retrieve stocks for each index
sp500_stocks = list(stock_data.get_stocks_by_index('S&P 500'))
dowjones_stocks = list(stock_data.get_stocks_by_index('DOW JONES'))
nasdaq100_stocks = list(stock_data.get_stocks_by_index('NASDAQ 100'))

# Combine all stocks into a single list
us_stocks = sp500_stocks + dowjones_stocks + nasdaq100_stocks

# Extract unique ticker symbols
tickers = list({stock['symbol'] for stock in us_stocks})

In [35]:
import ftplib
import pandas as pd
from io import StringIO

def fetch_us_listed_tickers():
    ftp = ftplib.FTP('ftp.nasdaqtrader.com')
    ftp.login()
    ftp.encoding = 'utf-8'
    ftp.cwd('SymbolDirectory')

    files = ["nasdaqlisted.txt", "otherlisted.txt"]
    all_dfs = []

    for file in files:
        buffer = []
        ftp.retrlines(f"RETR {file}", buffer.append)
        content = "\n".join(buffer)
        
        df = pd.read_csv(StringIO(content), sep="|")
        df = df[:-1]  # remove footer row
        all_dfs.append(df)

    ftp.quit()

    # Combine and return a unified DataFrame with standardized columns
    combined_df = pd.concat(all_dfs, ignore_index=True)
    combined_df.rename(columns={'Symbol': 'ticker', 'Security Name': 'company_name'}, inplace=True)
    return combined_df[['ticker', 'company_name']].drop_duplicates()

In [24]:
import yfinance as yf
import time

def enrich_with_yfinance(df: pd.DataFrame, sleep_time: float = 0.5) -> pd.DataFrame:
    """
    Adds 'yf_company_name' and 'market_cap' columns to the DataFrame using yfinance.
    Original 'company_name' from NASDAQ remains untouched for reference.
    """
    enriched_data = []

    for _, row in df.iterrows():
        ticker = row['ticker']
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            yf_name = info.get('longName', 'N/A')
            market_cap = info.get('marketCap', 'N/A')
        except Exception as e:
            print(f"Error for {ticker}: {e}")
            yf_name = "Error"
            market_cap = None
        
        enriched_data.append({
            'ticker': ticker,
            'original_name': row['company_name'],
            'yf_company_name': yf_name,
            'market_cap': market_cap
        })

        time.sleep(sleep_time)  # avoid rate limiting

    return pd.DataFrame(enriched_data)

In [36]:
tickers_df = fetch_us_listed_tickers()


In [38]:
tickers_df = tickers_df.drop_duplicates(subset=['ticker'])  # Remove duplicates based on ticker

In [39]:
tickers_df.head()

,ticker,company_name
0,AACB,Artius II Acquisition Inc. - Class A Ordinary ...
1,AACBR,Artius II Acquisition Inc. - Rights
2,AACBU,Artius II Acquisition Inc. - Units
3,AACG,ATA Creativity Global - American Depositary Sh...
4,AADR,AdvisorShares Dorsey Wright ADR ETF


In [26]:
enriched_df = enrich_with_yfinance(tickers_df)

500 Server Error: Internal Server Error for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LUCYW?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LUCYW&crumb=fhSyczRYun3


Error for LUCYW: 'NoneType' object has no attribute 'update'
Error for nan: 'float' object has no attribute 'upper'


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ZAZZT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ZAZZT&crumb=fhSyczRYun3


Error for ZAZZT: 'NoneType' object has no attribute 'update'


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ZBZZT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ZBZZT&crumb=fhSyczRYun3


...

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ZCZZT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ZCZZT&crumb=fhSyczRYun3


Error for ZCZZT: 'NoneType' object has no attribute 'update'


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ZXYZ.A?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ZXYZ.A&crumb=fhSyczRYun3


Error for ZXYZ.A: 'NoneType' object has no attribute 'update'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Err

KeyboardInterrupt: 

In [22]:
def filter_valid_companies(df: pd.DataFrame):
    """
    Filters the DataFrame to keep only rows with valid company names and market cap.
    """
    return df[
        (df['yf_company_name'].notna()) &
        (df['yf_company_name'] != 'N/A') &
        (df['yf_company_name'] != 'Error') &
        (df['market_cap'] != 'N/A')
    ].reset_index(drop=True)

In [ ]:
enriched_df

100

In [23]:
filtered_df = filter_valid_companies(enriched_df)
filtered_df.head()

,ticker,original_name,yf_company_name,market_cap
0,AACB,Artius II Acquisition Inc. - Class A Ordinary ...,Artius II Acquisition Inc. Class A Ordinary Sh...,248854880
1,AACBU,Artius II Acquisition Inc. - Units,Artius II Acquisition Inc. Units,253260512
2,AACG,ATA Creativity Global - American Depositary Sh...,ATA Creativity Global,30211798
3,AAL,"American Airlines Group, Inc. - Common Stock",American Airlines Group Inc.,6358759936
4,AAME,Atlantic American Corporation - Common Stock,Atlantic American Corporation,26931038


In [ ]:
filtered_df.to_csv('filtered_us_companies.csv', index=False)